# Name : Hassan Khan
# Task : 4

In [1]:
!pip install transformers
!pip install torch
!pip install datasets

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from datasets import load_dataset

In [3]:
# Load the dataset
dataset = load_dataset('conll2003')

# Display a sample
print(dataset['train'][0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [6]:
model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
# Create an NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)


In [8]:
# Example sentence
sentence = "Hugging Face Inc. is a company based in New York City. It is led by CEO Clément Delangue."

# Perform NER
ner_results = ner_pipeline(sentence)

# Display results
for entity in ner_results:
    print(f"Entity: {entity['word']}, Label: {entity['entity']}, Score: {entity['score']:.2f}")


Entity: Hu, Label: I-ORG, Score: 1.00
Entity: ##gging, Label: I-ORG, Score: 0.97
Entity: Face, Label: I-ORG, Score: 1.00
Entity: Inc, Label: I-ORG, Score: 1.00
Entity: New, Label: I-LOC, Score: 1.00
Entity: York, Label: I-LOC, Score: 1.00
Entity: City, Label: I-LOC, Score: 1.00
Entity: C, Label: I-PER, Score: 1.00
Entity: ##lé, Label: I-PER, Score: 1.00
Entity: ##ment, Label: I-PER, Score: 1.00
Entity: Del, Label: I-PER, Score: 1.00
Entity: ##ang, Label: I-PER, Score: 0.99
Entity: ##ue, Label: I-PER, Score: 1.00


In [10]:
#Preprecess the Dataset Files
def read_conll_file(file_path):
    with open(file_path, 'r') as file:
        sentences = []
        sentence = []
        for line in file:
            if line.strip() == "" or line.startswith("-DOCSTART-"):
                if sentence:
                    sentences.append(sentence)
                    sentence = []
            else:
                sentence.append(line.strip().split())
        if sentence:
            sentences.append(sentence)
    return sentences

# Replace with your local paths
train_data = read_conll_file('train.txt')
test_data = read_conll_file('test.txt')
dev_data = read_conll_file('dev.txt')

# Perform NER on the loaded data
for sentence in train_data[:5]:  # Only process the first 5 sentences for demonstration
    words = [word[0] for word in sentence]
    joined_sentence = " ".join(words)
    ner_results = ner_pipeline(joined_sentence)
    print(ner_results)


[{'entity': 'I-ORG', 'score': 0.9985098, 'index': 1, 'word': 'EU', 'start': 0, 'end': 2}, {'entity': 'I-MISC', 'score': 0.99920005, 'index': 3, 'word': 'German', 'start': 11, 'end': 17}, {'entity': 'I-MISC', 'score': 0.9993913, 'index': 7, 'word': 'British', 'start': 34, 'end': 41}]
[{'entity': 'I-PER', 'score': 0.9996301, 'index': 1, 'word': 'Peter', 'start': 0, 'end': 5}, {'entity': 'I-PER', 'score': 0.9997451, 'index': 2, 'word': 'Blackburn', 'start': 6, 'end': 15}]
[{'entity': 'I-LOC', 'score': 0.9979233, 'index': 1, 'word': 'BR', 'start': 0, 'end': 2}, {'entity': 'I-LOC', 'score': 0.9954905, 'index': 2, 'word': '##US', 'start': 2, 'end': 4}, {'entity': 'I-LOC', 'score': 0.99636036, 'index': 3, 'word': '##SE', 'start': 4, 'end': 6}, {'entity': 'I-LOC', 'score': 0.99703884, 'index': 4, 'word': '##LS', 'start': 6, 'end': 8}]
[{'entity': 'I-ORG', 'score': 0.9991308, 'index': 2, 'word': 'European', 'start': 4, 'end': 12}, {'entity': 'I-ORG', 'score': 0.99905294, 'index': 3, 'word': 'Co